In [ ]:
%pylab
import pandas as pd
import random
import os
from subprocess import Popen, PIPE
import subprocess

from scipy.optimize import differential_evolution
import numpy as np

In [ ]:
torcs_kill = 'killall torcs'
driver_cmd = 'java -cp .:torcs.jar main.Client --port:3002 --maxEpisodes:1'

race_xml = 'quickrace.xml'

trackTemplate = '<section name="%%IDX%%"><attstr name="name" val="%%TRACK_NAME%%"/><attstr name="category" val="%%TRACK_TYPE%%"/></section>'

tracks = [("dirt-1","dirt"),("dirt-2","dirt"),("dirt-3","dirt"),("dirt-4","dirt"),("dirt-5","dirt"),("dirt-6","dirt"),("mixed-1","dirt"),
          ("mixed-2","dirt"),("b-speedway","oval"),("c-speedway","oval"),("d-speedway","oval"),("e-speedway","oval"),("e-track-5","oval"),
          ("f-speedway", "oval"),("g-speedway","oval"),("michigan","oval"),("aalborg","road"),
          ("brondehach","road"),("g-track-1","road"),("ole-road-1","road"),("e-track-4","road"),("street-1","road"),("alpine-1","road"),
          ("corkscrew","road"),("e-track-2","road"),("e-track-6","road"),("g-track-2","road"),
          ("ruudskogen","road"),("wheel-1","road"),("alpine-2","road"),("eroad","road"),
          ("e-track-3","road"),("forza","road"),("g-track-3","road"),("wheel-2","road")]

In [ ]:
def getTrackCmd(i, trackName, trackType):
    return trackTemplate.replace("%%TRACK_NAME%%", trackName).replace("%%TRACK_TYPE%%", trackType).replace('%%IDX%%', str(i+1))

def generateXML():
    path = "./track.xml.tmpl"

    tracks_xml = ''
    picks = np.random.choice(range(len(tracks)), 1, replace=False)
    for i,t in enumerate(picks):
        tracks_xml += getTrackCmd(i, tracks[t][0], tracks[t][1]) + '\n'


    with open(path, "r") as f:
        content = f.read()
    f.close()

    content = content.replace('%%TRACK%%', tracks_xml)

    with open(race_xml, "w") as f:
        f.write(content)
    f.close()


In [ ]:
def start_torcs():
    os.system(torcs_kill)
    
#     print "torcs -nofuel -nodamage -t 10000 -r %s" % os.path.join(os.getcwd(),race_xml)
    p = Popen('torcs -nofuel -nodamage -t 5000 -r /home/dana/Desktop/computational_intelligence/sociopathsubmission/torcs/python/quickrace.xml', 
          shell=True)
    

# start_torcs()
    

In [ ]:
def start_evaluation():

    generateXML()

#     print 'Starting TORCS'
    start_torcs()

    results = []
    
#     print 'Starting Controller'
    p = subprocess.Popen(driver_cmd.split(" "), stdout=subprocess.PIPE, 
                         cwd='/home/dana/Desktop/computational_intelligence/sociopathsubmission/torcs/out/artifacts/torcs_jar')
    out, err = p.communicate()
    
    if(err):
        print err

    usefulres = 0;
    for res in out:
        if 'LapPosition' in res:
            usefulres = res.split()[1]
            print usefulres

    return usefulres

In [ ]:
def evaluate_driver(car_params):
    # Creates car with input parameters, executes jar file and return average final racePos over 
    # randomly selected tracks as car fitness

    np.savetxt("./parameters.txt", np.array(car_params).reshape(-1,1).T, delimiter=",")

    return start_evaluation()

In [ ]:
# Sets search space for the car parameters. Chosen close to COBOSTAR and Jonas' params for fast convergence.
init_params = [ 50.0,2,125.0,240.0,5.0,10.0,0.0,0.0,8.0,0,2.6,0,1.0,0.7,1.0,1,0,10.0,0.392,0.150,117.5,123.6,34.56,2.03,73.3,150.0,-500.0,200.0,43.23,1.99,104.76,9.38,907.6,1.92,11.89,1.13,0.70,0.39,36.50,97.33,0.0,0.0,0.5,0.5,0.5,0.5]

print len(init_params)
# Sets boundaries within 10% range
bounds = []
for val in init_params:
    if val == 0:
        bounds.append((0.0,10.0))
    else:
        bounds.append((val*0.9, val*1.1))

# Runs differential evolution on the car parameters within specified boundaries
result = differential_evolution(evaluate_driver, bounds, strategy='best1bin',
                                maxiter=5, popsize=4, tol=0.01,
                                mutation=(0.5, 1), recombination=0.2, seed=None,
                                callback=None, disp=True, polish=True,
                                init='latinhypercube')

# Prints best found car parameters and objective function
print result